In [1]:
import numpy as np
import pandas as pd
# import torch
# import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pymorphy2
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [5]:
df = pd.read_csv('tweets.csv', encoding='latin-1' , names = ['toxic','2','3','4','5','text'])
df.head(10)

,toxic,2,3,4,5,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [6]:
df = df[['toxic','text']]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   toxic   1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [7]:
df.loc[df["toxic"]==0, "toxic"] = 1
df.loc[df["toxic"]==4, "toxic"] = 0

In [8]:
df

,toxic,text
0,1,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,is upset that he can't update his Facebook by ...
2,1,@Kenichan I dived many times for the ball. Man...
3,1,my whole body feels itchy and like its on fire
4,1,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,0,Just woke up. Having no school is the best fee...
1599996,0,TheWDB.com - Very cool to hear old Walt interv...
1599997,0,Are you ready for your MoJo Makeover? Ask me f...
1599998,0,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
y = df['toxic']
corpus = df['text']

In [10]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return " ".join(res)

def clear_text(text):
    pattern = r'[^a-zA-Z]'
    cyrillic_text = re.sub(pattern, " ", text)
    
    return " ".join(cyrillic_text.split())

In [11]:
lemmatized_text = corpus.apply(lambda x: lemmatize(clear_text(x)))

In [12]:
lemmatized_text

0          switchfoot http twitpic com y zl awww that s a...
1          is upset that he can t update his facebook by ...
2          kenichan i dived many times for the ball manag...
3             my whole body feels itchy and like its on fire
4          nationwideclass no it s not behaving at all i ...
                                 ...                        
1599995    just woke up having no school is the best feel...
1599996    thewdb com very cool to hear old walt intervie...
1599997    are you ready for your mojo makeover ask me fo...
1599998    happy th birthday to my boo of alll time tupac...
1599999    happy charitytuesday thenspcc sparkscharity sp...
Name: text, Length: 1600000, dtype: object

In [13]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_text, y)

In [14]:
count_vect = CountVectorizer(stop_words=stopwords)
n_gramm_train = count_vect.fit_transform(X_train)
n_gramm_test = count_vect.transform(X_test)

In [15]:
LR = LogisticRegression(max_iter=1000)
LR.fit(n_gramm_train, y_train)
print("\nF1:", f1_score(y_test, LR.predict(n_gramm_test)))


F1: 0.7768371991369606


In [ ]:
# 1. SGD Classifier (SGD) - Линейный классификатор с SGD-обучением (stochastic gradient descent - стохастический градиентный спуск)
# 2. Support Vector Machines (SVM) - Метод опорных векторов (kernel = 'linear')
# 3. Random Forest Classifier (RF) - Случайный лес (используются деревья решений)
# 4. Gaussian process classification (GP) - Гауссовская классификация (основана на аппроксимации Лапласа)
# 5. AdaBoost (Adaptive Boosting) Classifier (AB) - Адаптивное усиление
# 6. Decision tree classifier (DT) - Дерево решений (http://scikit-learn.org/stable/modules/tree.html)
# 7. Logistic Regression (LR) - Логистическая регрессия
# 8. Gaussian Naive Bayes (NB) - Гауссовский наивный байесовский классификатор
# 9. Support Vector (SV) Classification - Метод опорных векторов http://scikit-learn.org/stable/modules/svm.html#svm-classification
# 10. MLP (Multi-layer Perceptron) Classifier (MLP) - Многослойный перцептрон
# 11. K-Nearest Neighbors (KNN) - Метод K-ближайших соседей
# 12. Quadratic Discriminant Analysis (QDA) - Квадратичный дискриминантный анализ
# 13. Linear Discriminant Analysis (LDA) - Линейный дискриминантный анализ

In [17]:
classifiers = []
classifiers.append(('SGD', SGDClassifier(max_iter = 1500, tol = 1e-4)))
# classifiers.append(('SVL', SVC(kernel = 'linear', C = 0.025, probability = True))) # C - штраф в случае ошибки
classifiers.append(('RF', RandomForestClassifier(max_depth = 5, n_estimators = 10, max_features = 1)))
# classifiers.append(('GP', GaussianProcessClassifier()))
classifiers.append(('AB', AdaBoostClassifier()))
classifiers.append(('DT', DecisionTreeClassifier()))
classifiers.append(('LR', LogisticRegression(solver = 'lbfgs', max_iter = 500, multi_class = 'auto')))
classifiers.append(('NB', GaussianNB()))
classifiers.append(('SVR', SVC(gamma = 2, C = 1.0))) # gamma - коэффициент ядра для 'rbf' - radial basis function, 'poly' and 'sigmoid'
classifiers.append(('MLP', MLPClassifier(alpha = 0.01, max_iter = 200, solver = 'lbfgs', tol = 0.001)))
classifiers.append(('KNN', KNeighborsClassifier(3)))
classifiers.append(('QDA', QuadraticDiscriminantAnalysis()))
classifiers.append(('LDA', LinearDiscriminantAnalysis()))

In [16]:
DT =DecisionTreeClassifier()
NB =GaussianNB()
SVR = SVC(gamma = 2, C = 1.0)
MLP = MLPClassifier(alpha = 0.01, max_iter = 200, solver = 'lbfgs', tol = 0.001)
KNN = KNeighborsClassifier(3)
QDA = QuadraticDiscriminantAnalysis()
LDA = LinearDiscriminantAnalysis()

In [17]:
DT.fit(n_gramm_train, y_train)
print('DT',"\nF1:", f1_score(y_test, DT.predict(n_gramm_test)))

KeyboardInterrupt: 

In [ ]:
NB.fit(n_gramm_train, y_train)
print('NB',"\nF1:", f1_score(y_test, NB.predict(n_gramm_test)))

In [ ]:
MLP.fit(n_gramm_train, y_train)
print('MLP',"\nF1:", f1_score(y_test, MLP.predict(n_gramm_test)))

In [ ]:
KNN.fit(n_gramm_train, y_train)
print('KNN',"\nF1:", f1_score(y_test, KNN.predict(n_gramm_test)))

In [ ]:
QDA .fit(n_gramm_train, y_train)
print('QDA ',"\nF1:", f1_score(y_test, QDA.predict(n_gramm_test)))

In [ ]:
LDA.fit(n_gramm_train, y_train)
print('LDA ',"\nF1:", f1_score(y_test, LDA.predict(n_gramm_test)))

In [52]:
# Оценка методов
results = []
names = []
k = 10
for name, classifier in classifiers:
    # Генерируем индексы для выделения обучающих и тестовых данных
    # k - 1 подвыборок будут использованы для обучения, а одна - для проверки (валидации)
    kfold = model_selection.KFold(n_splits = k, random_state = 348)
    ## cv - стратегия кросс-валидации
    cv_results = model_selection.cross_val_score(classifier, lemmatized_text, y, cv = kfold, scoring = 'accuracy')
    results.append(cv_results)
    names.append(name)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))
print(name, np.round(cv_results, 3), 'Средняя точность: ', round(cv_results.mean(), 3))

LDA: nan (nan)
LDA [nan nan nan nan nan nan nan nan nan nan] Средняя точность:  nan


In [ ]:
for name, classifier in classifiers:
    classifier.fit(n_gramm_train, y_train)
    print(name,"\nF1:", f1_score(y_test, classifier.predict(n_gramm_test)))

SGD 
F1: 0.7559589702058063
RF 
F1: 5.9927188466013795e-05
AB 
F1: 0.5351016681882313
